In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.wkt import loads
from shapely import wkt
import folium
from branca.colormap import linear
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FixedLocator

In [ ]:
#Shapefile baixado em: https://www.ibge.gov.br/geociencias/organizacao-do-territorio/malhas-territoriais/15774-malhas.html

file_name = "2021-Q04.parquet"
df = pd.read_parquet(file_name)


df.rename(columns={'tile': 'geometry'}, inplace=True)

df['geometry'] = df['geometry'].apply(wkt.loads)

df_geo = gpd.GeoDataFrame(df, geometry='geometry')
df_geo.crs = "EPSG:4326"

df_geo.head()

In [ ]:
#shapefile_sp = gpd.read_file('SP_Municipios_2022.zip') 
shapefile_br = gpd.read_file('BR_Municipios_2022.zip') #Tentar rodar com brasil
distritos = shapefile_br.to_crs(4326)
distritos.head()

#nome_arquivo_excel = "distritos.xlsx"
#distritos.to_excel(nome_arquivo_excel, index=False)
#print(f"Arquivo '{nome_arquivo_excel}' salvo com sucesso!")

In [ ]:
# Realiza o join espacial
join_distritos = gpd.sjoin(df_geo, distritos, how="inner", op='intersects')

join_distritos.head()

In [ ]:
join_distritos['avg_d_mbps'] = join_distritos['avg_d_kbps'] / 1000
join_distritos['avg_u_mbps'] = join_distritos['avg_u_kbps'] / 1000

In [ ]:
# Dicionário de correspondência entre siglas e nomes completos
uf_dict = {
    'AC': 'Acre', 'AL': 'Alagoas', 'AM': 'Amazonas', 'AP': 'Amapá', 'BA': 'Bahia',
    'CE': 'Ceará', 'DF': 'Distrito Federal', 'ES': 'Espírito Santo', 'GO': 'Goiás',
    'MA': 'Maranhão', 'MG': 'Minas Gerais', 'MS': 'Mato Grosso do Sul', 'MT': 'Mato Grosso',
    'PA': 'Pará', 'PB': 'Paraíba', 'PE': 'Pernambuco', 'PI': 'Piauí', 'PR': 'Paraná',
    'RJ': 'Rio de Janeiro', 'RN': 'Rio Grande do Norte', 'RO': 'Rondônia', 'RR': 'Roraima','RS': 'Rio Grande do Sul',
    'SC': 'Santa Catarina', 'SE': 'Sergipe', 'SP': 'São Paulo', 'TO': 'Tocantins'
}
# Criando a nova coluna com os nomes completos das UFs
join_distritos['NOME_UF'] = join_distritos['SIGLA_UF'].map(uf_dict)

In [ ]:
df_idh = pd.read_excel('Atlas - IDH Brasileiro.xlsx')
#df_idh['Codigo_Municipio'] = df_idh['Codigo_Municipio'].astype(str)

In [ ]:
print(df_idh.head())

In [ ]:
# Realiza o join dos DataFrames
join_all = pd.merge(join_distritos, df_idh, left_on='NOME_UF', right_on='Territorialidade', how='left')
join_all = gpd.GeoDataFrame(join_all, geometry='geometry')
join_all = join_all.set_geometry('geometry')
join_all.info()


In [ ]:
print(join_all.columns)  # This will list all the columns in the DataFrame


In [ ]:
df_completo = join_all.dissolve(by='NOME_UF')


In [ ]:
gdf = gpd.GeoDataFrame(join_all, geometry='geometry')
gdf = gdf.set_geometry('geometry')
df_completo = gdf.dissolve(by='NOME_UF', aggfunc={
    'AREA_KM2': 'max',
    'avg_d_mbps': 'mean',
    'avg_u_mbps': 'mean',
    'devices': 'sum',
    'tests': 'sum',
    'Posição IDHM': 'max',
    'IDHM': 'max',
    'Posição IDHM Renda': 'max',
    'IDHM Renda': 'max',
    'Posição IDHM Educação': 'max',
    'IDHM Educação': 'max',
    'Posição IDHM Longevidade': 'max',
    'IDHM Longevidade': 'max'
})



In [ ]:
df_completo.reset_index(inplace=True)

#df_completo['geometry'] = df_completo.buffer(0)

print(df_completo.geom_type.unique()) # ['MultiPolygon'] esperado
df_completo.info()

In [ ]:
# Renomeia as colunas para nomes mais amigáveis
df_completo = df_completo.rename(columns={
    'NM_MUN': 'Município',
    'SIGLA_UF': 'UF',
    'AREA_KM2': 'Área (km²)',
    'avg_d_mbps': 'Velocidade Média Download (Mbps)',
    'avg_u_mbps': 'Velocidade Média Upload (Mbps)',
    'devices': 'Número de Dispositivos',
    'tests': 'Número de Testes',
})

print(df_completo.geom_type.unique())# ['MultiPolygon'] esperado
df_completo.info()

In [ ]:
nome_arquivo_excel_final = "dados_final_agregados.xlsx"
df_completo.to_excel(nome_arquivo_excel_final, index=False)
print(f"Arquivo '{nome_arquivo_excel_final}' salvo com sucesso!")

In [ ]:
def plot(df, dfxlabel, dfylabel, dftitle, limit):
    # Ordena os dados e limita ao número de elementos especificado
    df = df.nlargest(limit, dfylabel)
    
    x = df[dfxlabel].values
    y = df[dfylabel].values
    
    plt.figure(figsize=(50, 7))
    sns.set_style('darkgrid')
    plt.xlabel("", fontsize=22)
    plt.ylabel("", fontsize=22)
    plt.title(dftitle, fontsize=34)
    plt.ylim(min(y) - 5, max(y) + 5)
    plt.xticks(fontsize=22)
    plt.yticks(fontsize=22)

    # Create the barplot and get the plot object
    barplot = sns.barplot(x=x, y=y, color='MediumSeaGreen')

    # Get the x-axis labels from the plot object
    x_labels = barplot.get_xticklabels()

    # Set the x-ticks using FixedLocator
    barplot.xaxis.set_major_locator(FixedLocator(range(len(x_labels))))

    # Show the values for each bar
    for index, value in enumerate(y):
        plt.text(index, value - 0.05, str(round(value, 2)), color='black', ha="center", fontsize=22)  # Ajusta o tamanho do texto dos valores


    # Set the x-axis labels on the plot object
    lenght = 30
    barplot.set_xticklabels([label.get_text()[:lenght] + '...' if len(label.get_text()) > lenght else label.get_text() for label in x_labels], fontsize=22)  # Ajusta o tamanho do texto das unidades federativas
    
    plt.show()

In [ ]:
plot(df_completo, 'NOME_UF', 'IDHM', 'Top 10 IDHM', 10)

In [ ]:
# Top 10 municípios por número de dispositivos (Número de Dispositivos)
plot(df_completo, 'NOME_UF', 'Número de Dispositivos', 'Top 10 UF por Número de Dispositivos', 10)

In [ ]:
import folium

m = folium.Map(location=[-15.788497, -47.879873], zoom_start=4)

choropleth = folium.Choropleth(
    geo_data=df_completo.to_json(),  # Certifique-se de usar a conversão correta para JSON
    name='choropleth',
    data=df_completo,
    columns=['NOME_UF', 'Velocidade Média Download (Mbps)'],
    key_on='feature.properties.NOME_UF',  # Ajuste baseado em como os dados estão estruturados no GeoJSON
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name='Velocidade Média de Internet (Mbps)'
).add_to(m)

#m.save('mapa_brasil.html')

m

In [ ]:
import folium
import geopandas as gpd

# Supondo que df_completo já foi definido e é um GeoDataFrame com a coluna 'geometry' como polígonos dos estados
gdf = df_completo

# Iniciando um mapa Folium centrado no Brasil
m = folium.Map(location=[-14.2350, -51.9253], zoom_start=4)

# Função para aplicar a cor de acordo com o valor do IDHM
def color_producer(idhm):
    if idhm < 0.6:
        return 'red'
    elif idhm < 0.7:
        return 'orange'
    else:
        return 'green'

# Adicionando os polígonos ao mapa com a cor baseada no IDHM
for _, r in gdf.iterrows():
    folium.GeoJson(
        data=r['geometry'],
        style_function=lambda x, color=color_producer(r['IDHM']): {
            'fillColor': color,
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.6
        }
    ).add_to(m).add_child(folium.Popup(f"Estado: {r['NOME_UF']}<br>"
                                       f"Velocidade Média de Download: {r['Velocidade Média Download (Mbps)']} Mbps<br>"
                                       f"IDHM: {r['IDHM']}<br>"
                                       f"Número de Dispositivos: {r['Número de Dispositivos']}<br>"
                                       f"Número de Testes: {r['Número de Testes']}"))

# Salvando o mapa em um arquivo HTML
#m.save('mapa_dos_estados.html')

# Para visualizar o mapa no Jupyter Notebook, se estiver usando um:
m
